# 2025 DL Lab8: RL Assignment_Super Mario World

**Your Answer:**    
Hi I'm XXX, XXXXXXXXXX.

## Overview
This project implements a **Deep Reinforcement Learning** pipeline to train an autonomous agent for Super Mario World. Leveraging the **Proximal Policy Optimization (PPO)** algorithm, the system interacts with the **stable-retro** environment to master the YoshiIsland1 level. Key components include a custom Vision Backbone for extracting features from raw pixel data and a suite of Environment Wrappers that handle frame preprocessing, action discretization, and reward shaping to facilitate efficient learning.

Reward function implement  
should do something in the beginning (monster attack)  
Custom PPO implement  
pre train weight 差不多，主要是 reward function  
model weight capacity 1GB  
class name 不要動 (可以新增，但是原本有的不要動)

## Imports

In [1]:
import os
import numpy as np
import retro
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.vec_env import VecNormalize

from eval import evaluate_policy, record_video
from custom_policy import VisionBackbonePolicy, CustomPPO

/home/m314834001/Lab8/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Configuration

In [2]:
# Game Settings
GAME = "SuperMarioWorld-Snes"
STATE = "YoshiIsland1"

# Training Settings
# TOTAL_STEPS = 0x1400000 # 20,971,520
TOTAL_STEPS = 0x0A00000 # 10,485,760
# TOTAL_STEPS = 0X3200000 # 52,428,800
TRAIN_CHUNK = 0x0040000 #    262,144
N_ENVS = 16

# Evaluation & Recording Settings
EVAL_EPISODES = 3
EVAL_MAX_STEPS = 18000
RECORD_STEPS = 1200

# Directories
LOG_DIR = "./runs_smw"
VIDEO_DIR       = os.path.join(LOG_DIR, "videos")
CKPT_DIR        = os.path.join(LOG_DIR, "checkpoints")
TENSORBOARD_LOG = os.path.join(LOG_DIR, "tb")

os.makedirs(LOG_DIR,   exist_ok=True)
os.makedirs(CKPT_DIR,  exist_ok=True)
os.makedirs(VIDEO_DIR, exist_ok=True)

## Environment Functions

In [3]:
from wrappers import make_base_env
def _make_env_thunk(game: str, state: str):
    """Return a function that creates an environment (for multiprocessing)."""
    def _thunk():
        return make_base_env(game, state)
    return _thunk

def make_vec_env(game: str, state: str, n_envs: int, use_subproc: bool = True):
    """Create a vectorized environment (multiple envs running in parallel)."""
    env_fns = [_make_env_thunk(game, state) for _ in range(n_envs)]
    
    if use_subproc and n_envs > 1:
        vec_env = SubprocVecEnv(env_fns)
    else:
        vec_env = DummyVecEnv(env_fns)

    return vec_env

## Initialize Env & Model

In [4]:
# 1. Create Training Environment
train_env = make_vec_env(GAME, STATE, n_envs=N_ENVS)
# train_env = VecNormalize(train_env, norm_obs=True, norm_reward=True, clip_obs=10., clip_reward=10.)
print(f"Environment created: {GAME} - {STATE} with {N_ENVS} parallel envs.")

checkpoint_path = "None" # 6225920 (19) 有破壞
# checkpoint_path = "runs_smw/preserved/Enc5_67.zip"

best_mean = -1e18
trained = 0
round_idx = 0

# 2. Initialize Model
if os.path.exists(checkpoint_path):
    # 讀取現有模型
    model = CustomPPO.load(
        checkpoint_path, 
        env=train_env,
        device="cuda:0" # 確保使用 GPU
    )
    trained = model.num_timesteps
    round_idx = int(trained / TRAIN_CHUNK)
    print(f"[Sucess] Loaded model from {checkpoint_path}")
    print(f"trained: {trained}, round_index: {round_idx}")
else:
    print(f"[Fail] Can't load {checkpoint_path}. Will use new model")
    model = CustomPPO(
        VisionBackbonePolicy,
        train_env,
        policy_kwargs   = dict(normalize_images=False),
        n_epochs        = 4,
        n_steps         = 512,
        batch_size      = 512,
        learning_rate   = 1e-4,
        verbose         = 1,
        gamma           = 0.96875,
        kl_coef         = 1,
        clip_range      = 0.125,
        ent_coef        = 0.0375,
        tensorboard_log = TENSORBOARD_LOG,
    )

Environment created: SuperMarioWorld-Snes - YoshiIsland1 with 16 parallel envs.
[Fail] Can't load None. Will use new model
Using cuda:0 device


In [5]:
# model.save("policy")

In [6]:
# import os
# from custom_policy import CustomPPO
# from wrappers import make_base_env  # [新增] 必須引入這行來建立環境

# # ================= 設定區 =================
# # 請確保這些變數有被定義 (這裡沿用你原本的變數名稱)
# # GAME = "SuperMarioWorld-Snes"
# # STATE = "Level1" 
# # CKPT_DIR = "./"
# # RECORD_STEPS = 2000
# PSVD_DIR = "./runs_smw/preserved/"

# # target_numbers = list(range(159, 180))
# target_numbers = [124, 137, 147, 151, 179]

# # ================= 執行迴圈 =================
# print(f"準備測試以下 Checkpoints: {target_numbers}")

# for num in target_numbers:
#     model_path = os.path.join(PSVD_DIR, f"Enc5_{num}.zip")
    
#     # 檢查檔案是否存在
#     if not os.path.exists(model_path):
#         print(f"⚠️ 找不到檔案: {model_path}，跳過。")
#         continue
    
#     print(f"\n[{num}] 正在載入模型: {model_path} ...")
    
#     env = None
#     try:
#         # 1. 載入模型
#         model = CustomPPO.load(model_path, device="auto")
        
#         # 2. 建立環境 (這就是原本 record_video 裡做的事，但我們現在自己做)
#         env = make_base_env(game=GAME, state=STATE)
        
#         print(f"[{num}] 正在執行遊戲 (上限 {RECORD_STEPS} steps)...")
        
#         # 3. 開始試跑
#         obs, info = env.reset()
#         final_score = 0
        
#         for step in range(RECORD_STEPS):
#             # 預測動作
#             action, _ = model.predict(obs, deterministic=True)
            
#             # 執行動作
#             obs, reward, terminated, truncated, info = env.step(action)
            
#             # 更新當前分數 (如果有 score 的話)
#             if "score" in info:
#                 final_score = info["score"]
            
#             # 判斷是否結束
#             if terminated or truncated:
#                 print(f"   -> 遊戲在 step {step} 結束 (Terminated/Truncated)")
#                 break
        
#         # 4. 印出結果
#         print(f"🏆 Checkpoint {num} 最終分數: {final_score}")
        
#     except Exception as e:
#         print(f"❌ 發生錯誤 (Model: {num}): {e}")
        
#     finally:
#         # 確保環境被關閉，釋放資源
#         if env is not None:
#             env.close()

# print("\n所有測試結束。")

In [7]:
import os
import glob
from custom_policy import CustomPPO
from eval import record_video  # 確保 eval.py 在同一目錄下
PSVD_DIR = "./runs_smw/preserved/"
# ================= 設定區 =================
# target_numbers = list(range(38, 40))
target_numbers = [147, 151, 179]

# ================= 執行迴圈 =================
print(f"準備測試以下 Checkpoints: {target_numbers}")

for num in target_numbers:
    model_path = os.path.join(PSVD_DIR, f"Enc5_{num}.zip")
    
    # 檢查檔案是否存在
    if not os.path.exists(model_path):
        print(f"⚠️ 找不到檔案: {model_path}，跳過。")
        continue
    
    print(f"\n[{num}] 正在載入模型: {model_path} ...")
    
    try:
        # 1. 載入模型 (不需要 env 參數也能載入權重)
        # 如果你有改過 CustomPPO 的參數，load 會自動讀取 zip 裡的設定
        model = CustomPPO.load(model_path, device="auto") # device="auto" 會自動用 GPU
        
        # 2. 錄製影片
        prefix_name = f"test_{num}"
        print(f"[{num}] 正在錄影 (長度 {RECORD_STEPS} steps)...")
        
        record_video(
            model=model,
            game=GAME,
            state=STATE,
            out_dir=VIDEO_DIR,
            video_len=RECORD_STEPS,
            prefix=prefix_name
        )
        print(f"✅ 完成！影片已儲存為 {prefix_name}.mp4")
        
    except Exception as e:
        print(f"❌ 發生錯誤 (Model: {num}): {e}")

print("\n所有測試結束。")

準備測試以下 Checkpoints: [147, 151, 179]

[147] 正在載入模型: ./runs_smw/preserved/Enc5_147.zip ...
[147] 正在錄影 (長度 1200 steps)...
Saved video to ./runs_smw/videos/test_147.mp4
✅ 完成！影片已儲存為 test_147.mp4

[151] 正在載入模型: ./runs_smw/preserved/Enc5_151.zip ...
[151] 正在錄影 (長度 1200 steps)...
Saved video to ./runs_smw/videos/test_151.mp4
✅ 完成！影片已儲存為 test_151.mp4

[179] 正在載入模型: ./runs_smw/preserved/Enc5_179.zip ...
[179] 正在錄影 (長度 1200 steps)...
Saved video to ./runs_smw/videos/test_179.mp4
✅ 完成！影片已儲存為 test_179.mp4

所有測試結束。


## Training Loop

In [ ]:
try:
    while trained < TOTAL_STEPS:
        round_idx += 1
        chunk = min(TRAIN_CHUNK, TOTAL_STEPS - trained)
        # chunk = 2000
        label = "S2K" # smushed to killed
        tagged_label = f"{label}_{int(trained/TRAIN_CHUNK)}"

        print(f"\n=== Round {round_idx} | Learn {chunk} steps (Total trained: {trained}) ===")
        
        # --- Train ---
        model.learn(total_timesteps=chunk, reset_num_timesteps=False, tb_log_name=label)
        trained += chunk

        # --- Save Checkpoint ---
        ckpt_path = os.path.join(CKPT_DIR, f"{tagged_label}.zip")
        model.save(ckpt_path)
        print(f"Saved checkpoint: {ckpt_path}")

        # --- Evaluate ---
        mean_ret, best_ret = evaluate_policy(
            model,
            GAME,
            STATE,
            n_episodes=EVAL_EPISODES,
            max_steps=EVAL_MAX_STEPS,
        )
        print(f"[EVAL] Mean Return: {mean_ret:.3f}, Best Return: {best_ret:.3f}")

        # --- Save Best Model ---
        # if mean_ret > best_mean:
        #     best_mean = mean_ret
        #     best_path = os.path.join(LOG_DIR, "best_model.zip")
        #     model.save(best_path)
        #     print(f"New best record. Saved to {best_path}")

        # --- Record Video ---
        out_path = os.path.join(VIDEO_DIR, label)
        os.makedirs(out_path,  exist_ok=True)
        record_video(
            model,
            GAME,
            STATE,
            VIDEO_DIR,
            video_len=RECORD_STEPS,
            prefix=f"{label}/{tagged_label}_{mean_ret:.2f}",
        )

except KeyboardInterrupt:
    print("\nTraining interrupted manually.")

finally:
    train_env.close()
    print("Training finished. Environment closed.")
    
"""
tensorboard --logdir=./runs_smw/tb
"""

## Display Video

In [8]:
from IPython.display import Video
import glob
# label = "Dec22A"

# list_of_files = glob.glob(os.path.join(VIDEO_DIR, label, '*.mp4')) 
# if list_of_files:
#     latest_file = max(list_of_files, key=os.path.getctime)
#     print(f"Playing: {latest_file}")
#     latest_file = "runs_smw/videos/Dec22A/Dec22A_73_596.54.mp4"
#     print(f"Playing: {latest_file}")
#     display(Video(latest_file, embed=True, width=768))
# else:
#     print("No videos found yet.")
    
video = "./runs_smw/videos/test_147.mp4"
display(Video(video, embed=True, width=768))

In [ ]:
import cv2

cap = cv2.VideoCapture("runs_smw/videos/test_16.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    cv2.imshow("Frame-by-Frame", frame)

    # 關鍵：這裡等待按鍵。按 'n' 鍵跳到下一幀，按 'q' 離開
    key = cv2.waitKey(0) 
    if key == ord('q'):
        break
    elif key == ord('n'):
        continue

cap.release()
cv2.destroyAllWindows()